In [1]:
from components.fuzzymatching import FuzzyMatchingFactory
from core.config import Environment, Config


matching_tool_config = {'csv_path': './backend/db/keyframes.csv', 
                        'env_dir': Environment()}
ocr_matcher = FuzzyMatchingFactory.produce('rapidwuzzy', **matching_tool_config)


/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/octoopt/anaconda3/envs/hcmc/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [2]:
from components.llms import AgentFactory
from components.llms.prompts import EXTRACT_KEYWORDS

import os 
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm_config = {'api_key': GOOGLE_API_KEY}
llm_agent = AgentFactory.produce(provider='gemini', 
                                 **llm_config)
llm_agent

In [3]:
query = "Đoạn giới thiệu về lễ hội Việt - Nhật. Trong lễ hội có các phân cảnh những chiếc lồng đèn kiểu Nhật cùng những chiếc dù Wagasa. Phân đoạn giữa có hình ảnh của chú mèo máy Doraemon khổng lồ và có những người xung quanh chụp ảnh với nó. Phân đoạn sau đó có hình ảnh một ban nhạc Nhật chơi các nhạc cụ truyền thống trên sân khấu. Xuyên suốt đoạn giới thiệu là một số đoạn phỏng vấn những người tham gia lễ hội. Hỏi có tổng cộng bao nhiêu người khác nhau được phỏng vấn trong đoạn giới thiệu trên?"

In [4]:
import json

kw_v3 = llm_agent.run(EXTRACT_KEYWORDS + '\n' + query)
kw_v3 = json.loads(kw_v3)
kw_v3 = [d['keyword'] for d in kw_v3]
kw_v3

['Việt - Nhật festival',
 'Japanese lanterns',
 'Wagasa umbrellas',
 'Doraemon',
 'Japanese traditional music',
 'interview']

In [5]:
matching_paths_v3 = []

for kw in kw_v3: 
    result = ocr_matcher.run(kw)
    img_paths = ocr_matcher.get_image_paths(result)
    matching_paths_v3 += img_paths

print(len(matching_paths_v3))
matching_paths_v3 = list(set(matching_paths_v3))
print(len(matching_paths_v3))
print(matching_paths_v3)

60
51
['L02_V030-15045.webp', 'L18_V008-00318.webp', 'L24_V005-02088.webp', 'L03_V011-11207.webp', 'L24_V007-09780.webp', 'L24_V018-00000.webp', 'L24_V029-15532.webp', 'L24_V028-12507.webp', 'L10_V024-00419.webp', 'L24_V009-11008.webp', 'L10_V008-29727.webp', 'L09_V010-00657.webp', 'L05_V024-18145.webp', 'L24_V004-14444.webp', 'L20_V015-01702.webp', 'L21_V013-31966.webp', 'L24_V011-13801.webp', 'L24_V032-10966.webp', 'L24_V035-12168.webp', 'L24_V026-13456.webp', 'L14_V018-00036.webp', 'L09_V014-31190.webp', 'L19_V024-29772.webp', 'L24_V043-10213.webp', 'L23_V023-06165.webp', 'L24_V030-17737.webp', 'L24_V031-07816.webp', 'L24_V043-10716.webp', 'L20_V019-24400.webp', 'L21_V027-12655.webp', 'L24_V031-07865.webp', 'L24_V014-03893.webp', 'L24_V014-08476.webp', 'L01_V031-20472.webp', 'L20_V016-00007.webp', 'L10_V012-00034.webp', 'L24_V032-16779.webp', 'L17_V009-00042.webp', 'L24_V013-01155.webp', 'L01_V031-21436.webp', 'L24_V024-02104.webp', 'L09_V012-00005.webp', 'L24_V037-15045.webp', 'L24

In [6]:
import json
env_dir = Environment()
with open(os.path.join(env_dir.root, env_dir.db_root, env_dir.vid_url), 'r') as f:
    vid_url = json.load(f)

with open(os.path.join(env_dir.root, env_dir.db_root, env_dir.url_fps), 'r') as f:
    url_fps = json.load(f)

In [7]:
vid_urls = []
# embed_urls = []
frames = []
# image_paths = [image_path.split('/')[-1] for image_path in matching_paths]
for img_id in matching_paths_v3: 
    vid_id = img_id.split('.')[0]
    vid_name, frame = vid_id.split('-')
    url = vid_url[vid_name] + '&t=' + str(int(int(frame)/url_fps[vid_url[vid_name]]))
    # embed_url = vid_url[vid_name].replace('watch?v=', 'embed/') # TODO: make this execuate from frontend
    vid_urls.append(url)
    # embed_urls.append(embed_url)
    frames.append(frame)


In [8]:
from api.v1.searching import search_by_ocr

results = search_by_ocr(query=query, 
                        top_k=10, 
                        matching_tool=ocr_matcher, 
                        llm=llm_agent, 
                        vid_url=vid_url, 
                        url_fps=url_fps)
results

converting to json


{'image_paths': ['L20_V031-11320.webp',
  'L18_V015-16287.webp',
  'L12_V010-24105.webp',
  'L24_V004-12078.webp',
  'L24_V014-01304.webp',
  'L24_V014-01656.webp',
  'L14_V016-30491.webp',
  'L24_V006-08591.webp',
  'L24_V027-04205.webp',
  'L24_V009-11920.webp',
  'L04_V018-15273.webp',
  'L14_V027-01477.webp',
  'L24_V025-13055.webp',
  'L10_V008-29727.webp',
  'L02_V001-06644.webp',
  'L19_V024-10902.webp',
  'L20_V006-10307.webp',
  'L04_V002-20546.webp',
  'L24_V009-10654.webp',
  'L21_V013-31966.webp',
  'L24_V005-01295.webp',
  'L14_V001-18391.webp',
  'L24_V015-07970.webp',
  'L24_V021-08901.webp',
  'L20_V014-08215.webp',
  'L24_V043-05072.webp',
  'L19_V024-01141.webp',
  'L02_V001-12950.webp',
  'L09_V014-31190.webp',
  'L02_V016-00075.webp',
  'L24_V018-05249.webp',
  'L19_V024-29772.webp',
  'L24_V043-10213.webp',
  'L24_V020-08273.webp',
  'L16_V020-25845.webp',
  'L11_V011-26263.webp',
  'L24_V004-12111.webp',
  'L20_V019-24400.webp',
  'L09_V014-07325.webp',
  'L04_V00